![WireGuard Logo](https://www.wireguard.com/img/wireguard.svg)

### 1. Install/import necessary libraries
First, we need to install the WireGuard package on both the server and client to handle WireGuard configuration and key management.

```bash
# Install WireGuard package for managing VPN
sudo apt install wireguard
```

If we want to configure WireGuard using Python, we additionally need to install the `python-wireguard` package.

```bash
pip install python-wireguard
```

### 2. Generate WireGuard keys (as needed)
To establish a secure connection, we need to generate a public-private key pair on both the server and client:
- The private key remains secret on each device.
- The public key is shared with the other device to establish a secure connection.

Using bash:
```bash
# Generate private key and immediately pipe it to create the public key
wg genkey | tee privatekey | wg pubkey > publickey
```

Using Python with the `python-wireguard` library:
```python
# Import Key class from python_wireguard to generate keys in Python
from python_wireguard import Key

# Generate private and public key pair for WireGuard
private, public = Key.key_pair()
```

### 3. Set up WireGuard Server (or/and use a WireGuard test server)
Manual Configuration in `/etc/wireguard/wg0.conf`:

```bash
# WireGuard server configuration

[Interface]
# Server IP and subnet for VPN
Address = 10.0.0.1/24
ListenPort = 51820
PrivateKey = <server-private-key> # Replace with server's private key

[Peer]
PublicKey = <client-public-key> # Replace with client's public key
# Allow only the client IP to connect
AllowedIPs = 10.0.0.2/32
```

Using `python-wireguard` to configure the server:
```python
# Import necessary classes to set up server configuration in Python
from python_wireguard import Server, ClientConnection

# Create and enable WireGuard server with interface name, private key, IP, and port
server = Server("wg0", private, "10.0.0.1/24", 51820)
server.enable()

# Set up client connection with client's public key and IP
client_public_key = Key("<client-public-key>")
client_ip = "10.0.0.2"
conn = ClientConnection(client_public_key, client_ip)

# Add client connection to the server
server.add_client(conn)
```

### 4. Set up the client connection
Manual Configuration in `/etc/wireguard/wg0.conf`:

```bash
# WireGuard client configuration

[Interface]
# Client IP and subnet for VPN (needs to match the server's AllowedIPs)
Address = 10.0.0.2/24
PrivateKey = <client-private-key> 

[Peer]
PublicKey = <server-public-key> 
# Allowed IP range to be routed through the VPN tunnel (e.g., 0.0.0.0/0 for all traffic)
AllowedIPs = 10.0.0.1/32
# Server's public IP and the WireGuard listening port
Endpoint = 89.58.29.203:51820
# Keep the connection alive, especially if NAT or firewall is in use
PersistentKeepalive = 30
```

Using `python-wireguard` to configure the client:
```python
from python_wireguard import Client, ServerConnection, Key

# Create WireGuard client with interface name, private key, and IP (needs to match servers client_ip)
client = Client("wg0", private, "10.0.0.2/24")

# Define server connection details
server_public_key = Key("<server-public-key>")
server_ip = "89.58.29.203"
server_port = 51820

# Set up server connection in client configuration
server_conn = ServerConnection(server_public_key, server_ip, server_port)
client.set_server(server_conn)
```

### 5. Connect the client to the server (WireGuard)
Once both configurations are ready, start the WireGuard connection on both the server and the client:

Using Command Line (on both server and client):
```bash
# Bring up the WireGuard interface and establish the connection
wg-quick up wg0
```

Using `python-wireguard` (on the client):
```python
client.connect()
```

6. Caputer network packets

7. Save captured network packets

8. Analyze captured network packets

9. Visualize network packet data (e.g: packer protocol distribution)
    a. Filter packets for WireGuard
    b. Display filtered packet count

10. Create a dataframe to hold network packets information

11. Plots network packets lengths over time